In [1]:
#Los respectivos imports
import pandas as pd
import numpy as np
import json
from flare.data_science.features import domain_tld_extract
import caracteristicasDerivadas
from caracteristicasDerivadas import H_entropy, proporcionVocalesConsonantes, posicionPrimerDigito
import sklearn
from sklearn import metrics, model_selection, tree

Leyendo JSON

In [2]:
file_path = './large_eve.json'

data = []
for line in open(file_path, 'r'):
    data.append(json.loads(line))

print(len(data))

746909


Extrayendo datos con llave 'DNS'

In [3]:
data_dns = []

for j in data:
    for i in j:
        try: 
            if 'DNS' in i or 'dns' in i:
                data_dns.append(j)
        except:
            pass

print(data_dns[0], '\n', data_dns[1])

{'timestamp': '2017-07-22T17:33:16.661646-0500', 'flow_id': 1327836194150542, 'pcap_cnt': 22269, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 59680, 'dest_ip': '2001:0500:0001:0000:0000:0000:803f:0235', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 15529, 'rrname': 'api.wunderground.com', 'rrtype': 'A', 'tx_id': 0}} 
 {'timestamp': '2017-07-22T17:33:24.990320-0500', 'flow_id': 2022925111925872, 'pcap_cnt': 54352, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 38051, 'dest_ip': '2001:0500:0003:0000:0000:0000:0000:0042', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 58278, 'rrname': 'stork79.dropbox.com', 'rrtype': 'A', 'tx_id': 0}}


Normalizando JSON a pandas dataframe

In [4]:
df = pd.json_normalize(data_dns)

#Renombrando columnas nuevas
df = df.rename(columns= {'dns.type': 'dns_type', 'dns.id': 'dns_id', 'dns.rrname': 'dns_rrname', 'dns.rrtype': 'dns_rrtype', 'dns.tx_id': 'dns_tx_id', 'dns.rcode': 'dns_rcode', 'dns.ttl': 'dns_ttl', 'dns.rdata': 'dns_rdata'})

Filtrando registros con DNS tipo A

In [5]:
df_dns_A = df.query("dns_rrtype == 'A'")

print(df_dns_A.head())
print(len(df_dns_A))

                         timestamp           flow_id  pcap_cnt event_type  \
0  2017-07-22T17:33:16.661646-0500  1327836194150542     22269        dns   
1  2017-07-22T17:33:24.990320-0500  2022925111925872     54352        dns   
2  2017-07-22T17:33:27.379891-0500   578544790391795     54519        dns   
5  2017-07-22T17:33:36.672785-0500   237919524635665     55496        dns   
6  2017-07-22T17:33:38.537426-0500  2167545251640146     55687        dns   

   vlan                                   src_ip  src_port  \
0   110  2001:0dbb:0c18:0011:0260:6eff:fe30:0863     59680   
1   110  2001:0dbb:0c18:0011:0260:6eff:fe30:0863     38051   
2   150                          192.168.205.170     31393   
5   110  2001:0dbb:0c18:0011:0260:6eff:fe30:0863     41663   
6   180                           192.168.198.62     35092   

                                   dest_ip  dest_port proto dns_type  dns_id  \
0  2001:0500:0001:0000:0000:0000:803f:0235         53   UDP    query   15529   
1  2

Filtrando dominios unicos

In [6]:
df2 = df_dns_A.drop_duplicates(subset='dns_rrname', keep=False, ignore_index=True)

print(df2['dns_rrname'].value_counts())
print(len(df2))

portswigger.net                                 1
www.phpmyadmin.net                              1
aosnotify.me.com                                1
kodapp.com                                      1
ul.backblaze.com                                1
www.social-engineer.org                         1
activex.microsoft.com                           1
www.mac.com                                     1
gfe.nvidia.com                                  1
idisk.mac.com                                   1
www.nagios.org                                  1
vtlfccmfxlkgifuf.com                            1
linkhelp.clients.google.com.ad.aol.aoltw.net    1
update.macromates.com                           1
1922.168.22.254                                 1
linkhelp.clients.google.com                     1
1922.168.22.254.home                            1
192.168.21-28.0                                 1
ejfodfmfxlkgifuf.xyz                            1
192.168.21-28.0.home                            1


Extrayendo Top Level Domain

In [7]:
tld_list = []

print(df2.head())

for i in range(len(df2)):
    tld_list.append(domain_tld_extract(df2['dns_rrname'][i]))

df2['tld'] = tld_list



print(df2.head())

                         timestamp           flow_id  pcap_cnt event_type  \
0  2017-07-22T17:40:48.114275-0500  1014350855847523     96677        dns   
1  2017-07-22T17:46:04.610289-0500   429530932048697    151992        dns   
2  2017-07-22T17:48:23.254250-0500  1671304788304170    203830        dns   
3  2017-07-22T17:48:23.255644-0500  1964865802987164    203832        dns   
4  2017-07-22T17:53:44.404584-0500   959585778674792    234695        dns   

   vlan           src_ip  src_port        dest_ip  dest_port proto dns_type  \
0   140   192.168.204.63     52070  192.168.207.4         53   UDP    query   
1   110   192.168.201.65      1034  192.168.207.4         53   UDP    query   
2   120  192.168.202.172     55069  192.168.207.4         53   UDP    query   
3   120  192.168.202.172     54399  192.168.207.4         53   UDP    query   
4   120  192.168.202.172     49372  192.168.207.4         53   UDP    query   

   dns_id          dns_rrname dns_rrtype  dns_tx_id dns_rcode 

/tmp/ipykernel_10621/1648211025.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['tld'] = tld_list


Entrenar Modelo

In [27]:
#Carga de datos
df_m = pd.read_csv('./dga_data_small.csv')

#Preprocesamiento

#Eliminacion de caracteristicas irrelevante o repetidas
df_m.drop(['host','subclass'], axis=1, inplace=True)

#Codificacion de variable objetivo
df_m['isDGA'] = df_m['isDGA'].replace(to_replace='dga', value=1)
df_m['isDGA'] = df_m['isDGA'].replace(to_replace='legit', value=0)


#Derivar caracteristicas
# 1. Longitud
# 2. Cantidad de digitos
# 3. Calculo de entropia (Shannon), es decir la cantidad de "informacion" que se puede obtener de una variable
# 4. Posicion del primer digito en la cadena
# 5. Proporcion de vocales - consonantes

df_m['longitud'] = df_m['domain'].str.len()
df_m['digitos'] = df_m['domain'].str.count('[0-9]')
df_m['entropia'] = df_m['domain'].apply(H_entropy)
df_m['proporcionVocalesConsonantes'] = df_m['domain'].apply(proporcionVocalesConsonantes)
df_m['posicionPrimerDigito'] = df_m['domain'].apply(posicionPrimerDigito)

df_m.drop(['domain'], axis=1, inplace=True)



print('Final features:', df_m.columns)
print(df_m.head())

df_m_final = df_m

Final features: Index(['isDGA', 'longitud', 'digitos', 'entropia',
       'proporcionVocalesConsonantes', 'posicionPrimerDigito'],
      dtype='object')
   isDGA  longitud  digitos  entropia  proporcionVocalesConsonantes  \
0      1        13        0  3.546594                      0.083333   
1      1        26       10  4.132944                      0.333333   
2      1         8        0  2.500000                      0.333333   
3      1        26        7  4.180833                      0.357143   
4      1        24        9  3.834963                      0.666667   

   posicionPrimerDigito  
0                     0  
1                     1  
2                     0  
3                     1  
4                     2  


In [28]:
target = df_m_final['isDGA']
feature_matrix = df_m_final.drop(['isDGA'], axis=1)

print('Final features:', feature_matrix.columns)
feature_matrix.head()

feature_matrix_train, feature_matrix_test, target_train, target_test = model_selection.train_test_split(feature_matrix, target, test_size=0.5, random_state=31)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(feature_matrix_train, target_train)

Final features: Index(['longitud', 'digitos', 'entropia', 'proporcionVocalesConsonantes',
       'posicionPrimerDigito'],
      dtype='object')


Obtener data

In [29]:
d = {'longitud': [], 'digitos': [], 'entropia': [], 'proporcionVocalesConsonantes': [], 'posicionPrimerDigito': []}
df_t = pd.DataFrame(data=d)
df_t['longitud'] = df2['tld'].str.len()
df_t['digitos'] = df2['tld'].str.count('[0-9]')
df_t['entropia'] = df2['tld'].apply(H_entropy)
df_t['proporcionVocalesConsonantes'] = df2['tld'].apply(proporcionVocalesConsonantes)
df_t['posicionPrimerDigito'] = df2['tld'].apply(posicionPrimerDigito)

df_t_final = df_t
print(df_t_final.head())

   longitud  digitos  entropia  proporcionVocalesConsonantes  \
0        15        0  3.373557                      0.400000   
1        14        0  3.378783                      0.300000   
2        14        0  3.235926                      0.300000   
3        15        0  3.323231                      0.750000   
4        11        0  3.095795                      0.428571   

   posicionPrimerDigito  
0                     0  
1                     0  
2                     0  
3                     0  
4                     0  


Realizar Prediccion

In [36]:
target_pred = clf.predict(df_t_final)
print(target_pred)

[1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 1 0 0 1 0 1 1 1 1 1 0 0 0 0 0 1 0]


In [37]:
df2['isDGA'] = target_pred
print(df2.head())

                         timestamp           flow_id  pcap_cnt event_type  \
0  2017-07-22T17:40:48.114275-0500  1014350855847523     96677        dns   
1  2017-07-22T17:46:04.610289-0500   429530932048697    151992        dns   
2  2017-07-22T17:48:23.254250-0500  1671304788304170    203830        dns   
3  2017-07-22T17:48:23.255644-0500  1964865802987164    203832        dns   
4  2017-07-22T17:53:44.404584-0500   959585778674792    234695        dns   

   vlan           src_ip  src_port        dest_ip  dest_port proto dns_type  \
0   140   192.168.204.63     52070  192.168.207.4         53   UDP    query   
1   110   192.168.201.65      1034  192.168.207.4         53   UDP    query   
2   120  192.168.202.172     55069  192.168.207.4         53   UDP    query   
3   120  192.168.202.172     54399  192.168.207.4         53   UDP    query   
4   120  192.168.202.172     49372  192.168.207.4         53   UDP    query   

   dns_id          dns_rrname dns_rrtype  dns_tx_id dns_rcode 

/tmp/ipykernel_10621/3330053325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['isDGA'] = target_pred


In [ ]:
Registros que predijo son DGA

In [38]:
df_dga = df2[df2['isDGA']==1]

In [39]:
print(df_dga)

                          timestamp           flow_id  pcap_cnt event_type  \
0   2017-07-22T17:40:48.114275-0500  1014350855847523     96677        dns   
1   2017-07-22T17:46:04.610289-0500   429530932048697    151992        dns   
2   2017-07-22T17:48:23.254250-0500  1671304788304170    203830        dns   
3   2017-07-22T17:48:23.255644-0500  1964865802987164    203832        dns   
7   2017-07-22T17:48:23.252747-0500  1425258996816715    203828        dns   
23  2017-07-22T18:38:08.315492-0500  1181225593131108    595058        dns   
32  2017-07-22T18:51:27.204097-0500  1763090634841409   2163709        dns   
39  2017-07-22T19:08:35.866745-0500   455668379133712   3352456        dns   
42  2017-07-22T19:22:18.552958-0500  1206308375785470   2942239        dns   
44  2017-07-22T19:22:18.553308-0500   836902533624156   2942241        dns   
45  2017-07-22T19:27:13.025729-0500   777786623091841   3238835        dns   
46  2017-07-22T19:29:18.937879-0500   455668379133752   3352456 